In [1]:
import os, sys, re
import pandas as pd
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
from datetime import datetime, timedelta

current_directory = os.getcwd()
root_directory = os.path.abspath(os.path.join(current_directory, os.pardir))
sys.path.append(root_directory)

* **Concate all data**

In [2]:
import os
import pandas as pd

# Directory containing the CSV files
#directory = r'c:\Users\mohammed\OneDrive\Documents\QFM -S2\BankReviewIntelligence\ScrapperService\production_standalone\row_data\parcket\2024-06-08'
directory = "../../ScrapperService/production_standalone/row_data/parcket/2024-06-08"
# List to hold the data from each CSV file
data_frames = []

# Iterate over all files in the directory
for country in os.listdir(directory):
    if country != ".DS_Store":
        for city in os.listdir(directory+"/"+country):
            if city.endswith(".parquet"):
                file_path = os.path.join(directory+"/"+country, city)
                df = pd.read_parquet(file_path)
                data_frames.append(df)

# Concatenate all DataFrames
all_data = pd.concat(data_frames, ignore_index=True)
all_data['Reviewer_Publish_Date'] = all_data['Reviewer_Publish_Date'].str.replace('\xa0', ' ')

# Save the concatenated DataFrame to a new CSV file
all_data.to_parquet('concatenated_data.parquet', index=False)
print(all_data.shape)
#all_data.head()

(1710, 14)


* **Preprocessing**

In [3]:
from parquet_preprocessing import preprocess_dataframe
all_data = pd.read_parquet('concatenated_data.parquet')

In [4]:
df = preprocess_dataframe(all_data)

In [5]:
df

,Country,Town,Bank_Name,Bank_Phone_number,Bank_Address,Bank_Website,Reviewer_Nane,Reviewer_Sart,Reviewer_Text,Reviewer_Publish_Date,Reviewer_Like_Reaction,Reviewer_Profil_Link,Reviewer_Owner_Reply,Reviewer_Owner_Reply_Date,Topic,Sentiment,Sub_Topic
0,Kenya,Nyeri,Standard Chartered Bank Nyeri Branch,+254 20 3293900,"Kenyatta Rd, Nyeri, Kenya",standardchartered.com,V M,5,NAN,2018-06-29,0,https://www.google.com/maps/contrib/1169461831...,None,0000-00-00,NaN,NaN,NaN
1,Kenya,Kitui,Co-Op Bank Atm - Kitui Branch,+254 44 4422370,"Emco House, Kilungya Makuti Street, Kitui, Kenya",co-opbank.co.ke,Emman Emmah,5,NAN,2023-10-31,0,https://www.google.com/maps/contrib/1125754277...,None,0000-00-00,NaN,NaN,NaN
2,Kenya,Voi,Kcb Voi,Not available,"Voi, Kenya",not available,Isaac Akuru Miruka,5,NAN,2021-06-28,0,https://www.google.com/maps/contrib/1090182152...,None,0000-00-00,NaN,NaN,NaN
3,Kenya,Kitale,Standard Chartered Bank - Kitale Branch,+254 20 3293900,"Ground Floor- Teryet Business Centre, Kenyatta...",standardchartered.com,Tony Tonny,1,NAN,2020-06-28,0,https://www.google.com/maps/contrib/1018542844...,None,0000-00-00,NaN,NaN,NaN
4,Kenya,Homa Bay,"Barclays Bank, Homa Bay",Not available,"Fff4+7W4, Homa Bay, Kenya",not available,Declan Ottaro,3,Salle bancaire,2018-06-29,0,https://www.google.com/maps/contrib/1023346861...,None,0000-00-00,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1705,Côte D'Ivoire,Divo,Sgci,+225 51225122,"Rjrr+4W8, Divo, Côte D’Ivoire",not available,Marie Germaine,5,NAN,2019-06-29,0,https://www.google.com/maps/contrib/1054373247...,None,0000-00-00,NaN,NaN,NaN
1706,Côte D'Ivoire,Divo,Coopec Divo,Not available,"Rjmm+P5W, Divo, Côte D’Ivoire",not available,Comoe Djibril,2,NAN,2019-06-29,0,https://www.google.com/maps/contrib/1132578295...,None,0000-00-00,NaN,NaN,NaN
1707,Côte D'Ivoire,Divo,Coopec Divo,Not available,"Rjmm+P5W, Divo, Côte D’Ivoire",not available,Comoe Djibril,2,NAN,2019-06-29,0,https://www.google.com/maps/contrib/1132578295...,None,0000-00-00,NaN,NaN,NaN
1708,Côte D'Ivoire,Divo,Hamed & El Djo Transfert Divo,+225 05 56 33 4482,"Bp 385, Divo, Côte D’Ivoire",not available,Moctar Kone (Kmoctar),3,NAN,2023-06-28,0,https://www.google.com/maps/contrib/1036178797...,None,0000-00-00,NaN,NaN,NaN


* **Topic generation**

In [6]:
from llm.topificator import TopicExtractor
from tqdm import tqdm
extractor = TopicExtractor(model="llama3", patience=5)

In [7]:
df_macro = pd.DataFrame(columns=list(df.columns))
output_file = 'macro_llamma.csv'

In [8]:
save_interval = 1
row_accumulated = 0

In [9]:
def generate_static_topics_and_sentiments(stars):
    if stars == 1:
        return ("Expérience", "Negative", [])
    elif stars == 2:
        return ("Expérience", "Negative", [])
    elif stars == 3:
        return ("Expérience", "Neutral", [])
    elif stars == 4:
        return ("Expérience", "Positive", [])
    elif stars == 5:
        return ("Expérience", "Positive", [])
    else:
        return ("Experience", "Neutral", [])

In [10]:
# Itérer sur chaque ligne du DataFrame
for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    review = row['Reviewer_Text']
    
    try:
        topics = extractor.extract(review, type='SINGLE_SOURCE')
        topics_array = topics['topics']
    
        if len(topics_array) == 0:
            static_topic, static_sentiment, static_sub_topic = generate_static_topics_and_sentiments(row['Reviewer_Sart'])
            new_row = row.copy()
            new_row['Topic'] = static_topic
            new_row['Sentiment'] = static_sentiment
            new_row['Sub_Topic'] = static_sub_topic
            df_macro = pd.concat([df_macro, pd.DataFrame([new_row])], ignore_index=True)
            row_accumulated += 1
        else:
            for tuple_ in topics_array:
                new_row = row.copy()
                new_row['Topic'] = tuple_[0]
                new_row['Sentiment'] = tuple_[1]
                new_row['Sub_Topic'] = tuple_[2]
                df_macro = pd.concat([df_macro, pd.DataFrame([new_row])], ignore_index=True)
                row_accumulated += 1
            
        # Sauvegarder à chaque intervalle défini
        if row_accumulated >= save_interval:
            df_macro.to_csv(output_file, index=False)
            row_accumulated = 0
    except Exception as e:
        print(f"Erreur rencontrée à l'index {index}: {e}")
        pass

  1%|█▍                                                                                                       | 24/1710 [03:39<3:51:06,  8.22s/it]

Attempt 1 failed with error: Failed to parse response: No JSON object could be identified in the response
Attempt 2 failed with error: Failed to parse response: No JSON object could be identified in the response
Attempt 3 failed with error: Failed to parse response: No JSON object could be identified in the response
Attempt 4 failed with error: Failed to parse response: No JSON object could be identified in the response


  2%|██▏                                                                                                      | 35/1710 [05:01<3:38:55,  7.84s/it]

Attempt 1 failed with error: Failed to parse response: No JSON object could be identified in the response


 10%|██████████                                                                                              | 166/1710 [14:32<2:14:48,  5.24s/it]

...

 91%|█████████████████████████████████████████████████████████████████████████████████████████████▋         | 1556/1710 [3:11:37<35:04, 13.67s/it]

Attempt 1 failed with error: Failed to parse response: No JSON object could be identified in the response


 99%|██████████████████████████████████████████████████████████████████████████████████████████████████████▍| 1701/1710 [3:42:06<02:16, 15.20s/it]

Attempt 1 failed with error: Failed to parse response: No JSON object could be identified in the response


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 1710/1710 [3:44:50<00:00,  7.89s/it]


In [11]:
# Sauvegarder une dernière fois après la fin de la boucle
df_macro.to_csv(output_file, index=False)

# Afficher un message une fois le processus terminé
print(f"DataFrame mis à jour et sauvegardé dans {output_file}")

DataFrame mis à jour et sauvegardé dans macro_llamma.csv
